In [1]:
#numpy sklearn jieba gensim
import numpy as np
from sklearn.model_selection import train_test_split
import re
import string
import jieba
from sklearn.feature_extraction.text import CountVectorizer #载入词袋模型特征
from sklearn.feature_extraction.text import TfidfTransformer #载入Tfidf转换器
from sklearn.feature_extraction.text import TfidfVectorizer #载入Tfidf特征
import gensim
from sklearn.naive_bayes import MultinomialNB #导入多项朴素贝叶斯分类算法
from sklearn.linear_model import SGDClassifier #导入随机梯度下降分类器
from sklearn.linear_model import LogisticRegression #导入逻辑回归分类器
from sklearn import metrics

In [2]:
def get_data():
    """
    获取数据
    :return：文本数据，对应的labels
    """
    with open('./dataset/data/ham_data.txt', encoding='utf-8') as ham_f, open('./dataset/data/spam_data.txt',
                                                                              encoding='utf-8') as spam_f:
        ham_data = ham_f.readlines()
        spam_data = spam_f.readlines()

        ham_label = np.ones(len(ham_data)).tolist()
        spam_label = np.zeros(len(spam_data)).tolist()

        corpus = ham_data + spam_data

        labels = ham_label + spam_label

    return corpus, labels


def prepare_datasets(corpus, labels, test_data_proportion=0.3):
    """
    :param corpus:
    :param labels:
    :param test_data_proportion:
    :return:
    """
    train_x, test_x, train_y, test_y = train_test_split(corpus, labels, test_size=test_data_proportion, random_state=42)
    return train_x, test_x, train_y, test_y


def remove_empty_docs(corpus, labels):
    filtered_corpus = []
    filtered_labels = []
    for doc, label in zip(corpus, labels):
        if doc.strip():
            filtered_corpus.append(doc)
            filtered_labels.append(label)

    return filtered_corpus, filtered_labels


corpus, labels = get_data()

print(f'总的数据量：{len(labels)}')

corpus, labels = remove_empty_docs(corpus, labels)

print(f'样本之一：{corpus[10]}')
print(f'样本的label：{labels[10]}')
label_name_map = ['垃圾邮件', '正常邮件']
print(f'实际类型：{label_name_map[int(labels[10])]},'
      f'{label_name_map[int(labels[5900])]}')

# 对数据进行划分
train_corpus, test_corpus, train_labels, test_labels = prepare_datasets(corpus, labels, test_data_proportion=0.3)


总的数据量：10001
样本之一：北京售票员可厉害，嘿嘿，有专座的，会直接拉着脖子指着鼻子让上面的人站起来让 座的，呵呵，比较赞。。。 杭州就是很少有人给让座，除非司机要求乘客那样做。 五一去杭州一个景点玩，车上有两个不到一岁的小孩，就是没有人给让座，没办法家长只能在车上把小孩的推车打开让孩子坐进去，但是孩子还是闹，只能抱着，景点离市区很远，车上很颠，最后家长坐在地上抱孩子，就是没有一个人给让座，要是在北京，一上车就有人让座了

样本的label：1.0
实际类型：正常邮件,垃圾邮件


In [3]:
with open('./dataset/stop_words.utf8', encoding='utf-8') as f:
    stopword_list = f.readlines()


# jieba分词
def tokenize_text(text):
    tokens = jieba.lcut(text)
    tokens = [token.strip() for token in tokens]
    return tokens


# 去除特殊字符
def remove_special_characters(text):
    tokens = tokenize_text(text)
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_tokens = filter(None, [pattern.sub('', token) for token in tokens])
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text


# 去除停用词
def remove_stopwords(text):
    tokens = tokenize_text(text)
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text


# 当tokenize=True时候，会对文本进行分词，除此以外还会对文本进行去除特殊符号、停用词等预处理
def normalize_corpus(corpus, tokenize=False):
    normalized_corpus = []
    for text in corpus:
        text = remove_special_characters(text)
        text = remove_stopwords(text)
        normalized_corpus.append(text)
        if tokenize:
            text = tokenize_text(text)
            normalized_corpus.append(text)
    return normalized_corpus


In [4]:
def bow_extractor(corpus, ngram_range=(1, 1)):
    vectorizer = CountVectorizer(min_df=1, ngram_range=ngram_range)
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features


# Tfidf特征转换
def tfidf_transformer(bow_matrix):
    transformer = TfidfTransformer(norm='l2', smooth_idf=True, use_idf=True)
    tfidf_matrix = transformer.fit_transform(bow_matrix)
    return transformer, tfidf_matrix


# tfidf特征提取
def tfidf_extractor(corpus, ngram_range=(1, 1)):
    vectorizer = TfidfVectorizer(min_df=1, norm='l2', smooth_idf=True, use_idf=True, ngram_range=ngram_range)
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features


# 进行归一化
norm_train_corpus = normalize_corpus(train_corpus)
norm_test_corpus = normalize_corpus(test_corpus)

# 词袋模型特征
bow_vectorizer, bow_train_features = bow_extractor(norm_train_corpus)
bow_test_features = bow_vectorizer.transform(norm_test_corpus)

# tf-idf特征
tfidf_vectorizer, tfidf_train_features = tfidf_extractor(norm_train_corpus)
tfidf_test_features = tfidf_vectorizer.transform(norm_test_corpus)

# 训练数据tokenize化
tokenized_train = [jieba.lcut(text) for text in norm_train_corpus]
print(tokenized_train[2:10])
tokenized_test = [jieba.lcut(text) for text in norm_test_corpus]

# build word2vec模型
model = gensim.models.Word2Vec(tokenized_train, window=100, min_count=30, sample=1e-3)

mnb = MultinomialNB()
svm = SGDClassifier(loss='hinge')
lr = LogisticRegression()


# 首先声明了mnb, svm, lr 三个分类器，然后传入train_predict_evaluate_model函数中
def train_predict_evaluate_model(classifier, train_features, train_labels, test_features, test_labels):
    # build model
    classifier.fit(train_features, train_labels)
    # predict using model
    predictions = classifier.predict(test_features)
    # evaluate model prediction performance
    get_metrics(true_labels=test_labels, predicted_labels=predictions)
    return predictions


def get_metrics(true_labels, predicted_labels):
    print('Accuracy:', np.round(metrics.accuracy_score(true_labels, predicted_labels), 2))
    print('Precision:', np.round(metrics.precision_score(true_labels, predicted_labels, average='weighted'), 2))
    print('F1 Score:', np.round(metrics.f1_score(true_labels, predicted_labels, average='weighted'), 2))


Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/l9/llzdgt2s3hz2rfc6cjp7wgb40000gn/T/jieba.cache
Loading model cost 0.351 seconds.
Prefix dict has been built successfully.


[['10156', ' ', ' ', '说', ' ', ' ', '的', ' ', ' ', '呵呵', ' ', ' ', '标', ' ', ' ', '题', ' ', ' ', 'Re', ' ', ' ', '我', ' ', ' ', '要是', ' ', ' ', '你', ' ', ' ', '女朋友', ' ', ' ', '绝对', ' ', ' ', '跟', ' ', ' ', '你', ' ', ' ', '分手', ' ', ' ', 'Re', ' ', ' ', '昨晚', ' ', ' ', '猫', ' ', ' ', '又', ' ', ' ', '闹', ' ', ' ', '了', ' ', ' ', '一夜', ' ', ' ', '嗯', ' ', ' ', '，', ' ', ' ', '谁', ' ', ' ', '说', ' ', ' ', '我', ' ', ' ', '养猫', ' ', ' ', '是因为', ' ', ' ', '有', ' ', ' ', '爱心', ' ', ' ', '我', ' ', ' ', '跟', ' ', ' ', '谁', ' ', ' ', '急', ' ', ' ', '是', ' ', ' ', '哈', ' ', ' ', '，', ' ', ' ', '不是', ' ', ' ', '用', ' ', ' ', '爱心', ' ', ' ', '区分', ' ', ' ', '的', ' ', ' ', '喜欢', ' ', ' ', '宠物', ' ', ' ', '的', ' ', ' ', '就', ' ', ' ', '养', ' ', ' ', '，', ' ', ' ', '不', ' ', ' ', '喜欢', ' ', ' ', '的', ' ', ' ', '就', ' ', ' ', '不养', ' ', ' ', '呗', ' ', ' ', '卷卷', ' ', ' ', '，', ' ', ' ', '你', ' ', ' ', '搞', ' ', ' ', '成', ' ', ' ', '这', ' ', ' ', '副', ' ', ' ', '鬼', ' ', ' ', '样子', ' ', ' ', '，', ' ', '

In [5]:
print('基于词袋模型的多项朴素贝叶斯')
mnb_bow_predictions = train_predict_evaluate_model(classifier=mnb, train_features=bow_train_features,
                                                   train_labels=train_labels, test_features=bow_test_features,
                                                   test_labels=test_labels)

# 输出结果：
# 自己跑

# 基于词袋模型特征的逻辑回归
print('基于词袋模型特征的逻辑回归')
lr_bow_predictions = train_predict_evaluate_model(classifier=lr, train_features=bow_train_features,
                                                  train_labels=train_labels, test_features=bow_test_features,
                                                  test_labels=test_labels)
# 输出结果：
# 自己跑

# 基于词袋模型的支持向量机方法
print('基于词袋模型的支持向量机方法')
svm_bow_predictions = train_predict_evaluate_model(classifier=svm, train_features=bow_train_features,
                                                   train_labels=train_labels, test_features=bow_test_features,
                                                   test_labels=test_labels)

# 输出结果：
# 自己跑

# 基于tf-idf特征的多项朴素贝叶斯
print('基于tf-idf特征的多项朴素贝叶斯')
mnb_tfidf_predictions = train_predict_evaluate_model(classifier=mnb, train_features=tfidf_train_features,
                                                     train_labels=train_labels, test_features=tfidf_test_features,
                                                     test_labels=test_labels)

# 输出结果：
# 自己跑

# 基于tf-idf特征的逻辑回归
print('基于tf-idf特征的逻辑回归')
lr_tfidf_predictions = train_predict_evaluate_model(classifier=lr, train_features=tfidf_train_features,
                                                    train_labels=train_labels, test_features=tfidf_test_features,
                                                    test_labels=test_labels)

# 输出结果：
# 自己跑

# 基于tf-idf特征的支持向量机方法
print('基于tf-idf特征的支持向量机方法')
svm_tfidf_predictions = train_predict_evaluate_model(classifier=svm, train_features=tfidf_train_features,
                                                     train_labels=train_labels, test_features=tfidf_test_features,
                                                     test_labels=test_labels)


基于词袋模型的多项朴素贝叶斯
Accuracy: 0.99
Precision: 0.99
F1 Score: 0.99
基于词袋模型特征的逻辑回归
Accuracy: 0.99
Precision: 0.99
F1 Score: 0.99
基于词袋模型的支持向量机方法
Accuracy: 0.99
Precision: 0.99
F1 Score: 0.99
基于tf-idf特征的多项朴素贝叶斯
Accuracy: 0.99
Precision: 0.99
F1 Score: 0.99
基于tf-idf特征的逻辑回归
Accuracy: 0.99
Precision: 0.99
F1 Score: 0.99
基于tf-idf特征的支持向量机方法
Accuracy: 0.99
Precision: 0.99
F1 Score: 0.99


In [6]:
num = 0
for document, label, predicted_label in zip(test_corpus, test_labels, svm_tfidf_predictions):
    if label == 0 and predicted_label == 0:
        print('邮件类型：', label_name_map[int(label)])
        print('预测的邮件类型：', label_name_map[int(predicted_label)])
        print('文本：-')
        print(re.sub('\n', '', document))
        num += 1
        if num == 4:
            break

num = 0
for document, label, predicted_label in zip(test_corpus, test_labels, svm_tfidf_predictions):
    if label == 1 and predicted_label == 1:
        print('邮件类型：', label_name_map[int(label)])
        print('预测的邮件类型：', label_name_map[int(predicted_label)])
        print('文本：-')
        print(re.sub('\n', '', document))
        num += 1
        if num == 4:
            break


邮件类型： 垃圾邮件
预测的邮件类型： 垃圾邮件
文本：-
中信（国际）电子科技有限公司推出新产品： 升职步步高、做生意发大财、连找情人都用的上，详情进入 网  址:  http://www.usa5588.com/ccc 电话：020-33770208   服务热线：013650852999
邮件类型： 垃圾邮件
预测的邮件类型： 垃圾邮件
文本：-
您好！ 我公司有多余的发票可以向外代开！（国税、地税、运输、广告、海关缴款书）。 如果贵公司（厂）有需要请来电洽谈、咨询！ 联系电话: 013510251389  陈先生 谢谢 顺祝商祺!
邮件类型： 垃圾邮件
预测的邮件类型： 垃圾邮件
文本：-
如果您在信箱中不能正常阅读此邮件，请点击这里
邮件类型： 垃圾邮件
预测的邮件类型： 垃圾邮件
文本：-
以下不能正确显示请点此 IFRAME: http://bbs.ewzw.com/viewthread.php?tid=3790
邮件类型： 正常邮件
预测的邮件类型： 正常邮件
文本：-
分专业吧，也分导师吧 标  题: Re: 问一个：有人觉得自己博士能混毕业吗 当然很好混毕业了 : 博士读到快中期了，始终感觉什么都不会，文章也没发几篇好的，论文的架构也没有， : 一切跟刚上的时候没有区别。但是事实上我也很辛苦的找资料，做实验，还进公司实习过， : 现在感觉好失败，内心已经放弃了，打算混毕业，不知道过来人有什么高招，请指点一二。 --
邮件类型： 正常邮件
预测的邮件类型： 正常邮件
文本：-
三年的水木生活,版大一直都照着我 我正是在伊的关照下逐步逐步的迈入水车行列 可惜我刚来贵版,伊就要走了 留个小整,以作纪念 版大真是个好人 恩 robertwc (我不想要nick) 共上站 1289 次，发表过 48999 篇文章 上次在  [Wed Sep  7 14:26:34 2005] 从 [211.151.90.88] 到本站一游。 离线时间[因在线上或非常断线不详] 信箱：[  ] 生命力： 身份: [版主]。 目前在站上，状态如下：
邮件类型： 正常邮件
预测的邮件类型： 正常邮件
文本：-
谢谢你:) 可能是这个浮躁的社会促使人也浮躁道德丧失，所以现在的婚姻失败的很多。相爱的人不一定在婚姻中能相扶相伴走很远。不仅你周围，每个有点社会阅历的人都能举出身边一大堆